In [1]:
import json
import os
import ruff
import subprocess
from itertools import chain
import re
import logging
import ast
from typing import List, Optional
import pandas as pd
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

In [2]:
wiki_data_dir = Path("../data/raw/wiki")
news_data_dir = Path("../data/raw/news")
github_repos_dir = Path("../data/raw/github_repos")

wiki_data_dir

WindowsPath('../data/raw/wiki')

In [3]:
def clean_text(text: str) -> str:
    # Remove extra citations like [1], [2], [a], etc.
    text = re.sub(r'\[\s*[\w, ]+\s*\]', '', text)

    # Replace special characters with space
    text = text.replace("\u00A0", " ")  # non-breaking space
    text = text.replace("\t", " ")
    
    # Merge multiple space lines into one
    text = re.sub("\n\s*\n+", "\n\n", text)
    
    # Strip leading and trailing whitespace
    text = text.strip()
    
    return text

<>:10: SyntaxWarning: invalid escape sequence '\s'
<>:10: SyntaxWarning: invalid escape sequence '\s'
C:\Users\DELL\AppData\Local\Temp\ipykernel_4652\2664004039.py:10: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub("\n\s*\n+", "\n\n", text)


In [4]:
wiki_df = pd.read_json(wiki_data_dir / "wiki_demo.json")

wiki_df.head()

,title,sections,intro
0,Trí tuệ nhân tạo,"[{'header': 'Trí tuệ nhân tạo', 'content': 'Cá...",Trí tuệ nhân tạo ( TTNT ) ( tiếng Anh : Artifi...
1,Học máy,"[{'header': 'Học máy', 'content': 'Học máy có ...",Học máy hay máy học ( machine learning ) là mộ...
2,Học sâu,"[{'header': 'Học sâu', 'content': 'Mạng thần k...","Học sâu ( tiếng Anh : deep learning , còn gọi ..."
3,Xử lý ngôn ngữ tự nhiên,"[{'header': 'Các bước xử lý', 'content': 'Phân...",Xử lý ngôn ngữ tự nhiên ( natural language pro...
4,Thị giác máy tính,"[{'header': 'Thị giác máy tính', 'content': 'T...",Thị giác máy tính ( tiếng Anh : computer visio...


In [5]:
wiki_data = []
wiki_df.columns = ["Title", "Sections", "Intro"]

for _, row in wiki_df.iterrows():
    title = row["Title"]
    intro = row["Intro"]
    sections = row["Sections"]
    
    doc = f"Title: {title}\nIntro: {intro}\n"

    for section in sections:
        sec_header = section["header"]
        sec_content = section["content"]
        doc += f"Section Header: {sec_header}\nSection Content: {sec_content}\n"

    wiki_data.append(doc.strip())

len(wiki_data)

19

In [6]:
wiki_data_cleaned = [clean_text(item) for item in wiki_data]

wiki_data_cleaned[:3]

['Title: Trí tuệ nhân tạo\nIntro: Trí tuệ nhân tạo ( TTNT ) ( tiếng Anh : Artificial intelligence , viết tắt: AI ) là khả năng của các hệ thống máy tính thực hiện các nhiệm vụ liên quan đến trí thông minh của con người , như học tập , suy luận , giải quyết vấn đề , nhận thức và đưa ra quyết định . Đây là một lĩnh vực nghiên cứu thuộc khoa học máy tính , tập trung phát triển và nghiên cứu các phương pháp cùng phần mềm giúp máy móc có khả năng nhận thức môi trường xung quanh , sử dụng học tập và trí tuệ để thực hiện hành động nhằm tối đa hóa khả năng đạt được các mục tiêu đã định. \nSection Header: Trí tuệ nhân tạo\nSection Content: Các ứng dụng nổi bật của AI bao gồm công cụ tìm kiếm web tiên tiến (ví dụ: Google Tìm kiếm ); hệ thống đề xuất (được sử dụng bởi YouTube , Amazon và Netflix ); trợ lý ảo (ví dụ: Trợ lý Google , Siri và Alexa ); xe tự lái (ví dụ: Waymo ); công cụ sáng tạo và nội dung tạo sinh (ví dụ: mô hình ngôn ngữ và nghệ thuật AI ); cùng khả năng chơi và phân tích vượt trộ

In [7]:
news_df = pd.read_json(news_data_dir / "news_data.json")
news_df.columns = ["Title", "Sections"]

news_df.head()

,Title,Sections
0,Công an Hà Nội khuyến cáo người dân không tham...,[{'header': 'Để bảo đảm an toàn cho bản thân v...
1,Ngừng tiếp thu tàu bay tại một số Cảng hàng kh...,[{'header': 'Cục Hàng không Việt Nam vừa chỉ đ...
2,440 đại biểu ưu tú dự Đại hội đại biểu Đảng bộ...,"[{'header': 'Chiều ngày 27/9, Tỉnh ủy Lào Cai ..."
3,Ông Lê Tiến Châu giữ chức Bí thư Thành ủy Hải ...,[{'header': 'Sau gần 2 ngày làm việc Đại hội đ...
4,Động thổ làm nhà hỗ trợ bé mồ côi 5 tuổi bị lũ...,[{'header': 'Sau hơn 1 tháng Tạp chí Công dân ...


In [8]:
news_data = []

for _, row in news_df.iterrows():
    title = row["Title"]
    sections = row["Sections"]

    doc = f"Title: {title}\n"
    
    for section in sections:
        sec_header = section["header"]
        sec_content = section["content"]
        doc += f"Section Header: {sec_header}\nSection Content: {sec_content}\n"

    news_data.append(doc.strip())

len(news_data)

208

In [9]:
news_data_cleaned = [clean_text(item) for item in news_data]

news_data_cleaned[:3]

['Title: Công an Hà Nội khuyến cáo người dân không tham gia một giải chạy do chưa đủ điều kiện pháp lý\nSection Header: Để bảo đảm an toàn cho bản thân và tuân thủ các quy định của pháp luật, Công an Thành phố Hà Nội khuyến cáo người dân không tham gia Giải chạy "Mỹ Đình Half Marathon 2025" vào ngày 28/9/2025.\nSection Content: Giải chạy "Mỹ Đình Half Marathon 2025" dự kiến diễn ra vào ngày 28/9/2025 tại khu vực Sân vận động Quốc gia Mỹ Đình chưa đủ điều kiện pháp lý để tổ chức. Ảnh minh họa \nSection Header: Giải chạy "Mỹ Đình Half Marathon 2025" chưa được chấp thuận về mặt chuyên môn và chưa được chấp thuận chủ trương tổ chức\nSection Content: Công an thành phố Hà Nội thông tin, qua công tác nắm tình hình, phòng An ninh chính trị nội bộ Công an thành phố Hà Nội phát hiện giải chạy "Mỹ Đình Half Marathon 2025" dự kiến diễn ra vào ngày 28/9/2025 tại khu vực Sân vận động Quốc gia Mỹ Đình chưa đủ điều kiện pháp lý để tổ chức. Cụ thể đến thời điểm hiện tại, Công ty Cổ phần Di sản Văn hóa,

In [10]:
def read_python_files(repo_path):
    data = []
    
    for root, _, files in os.walk(repo_path):
        for file in files:
            if file.endswith(".py"):
                file_path = os.path.join(root, file)
                try:
                    if os.path.getsize(file_path) < 1e6:  # Skip files larger than 1MB
                        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                            content = f.read()
                        data.append({
                            "path": file_path,
                            "content": content
                        })
                except Exception as e:
                    print(f"Read error {file_path}: {e}")
    return data

def run_cmd(cmd: List[str]) -> subprocess.CompletedProcess:
    """Run subprocess and return CompletedProcess (capture output)."""
    return subprocess.run(cmd, capture_output=True, text=True)

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")
logger = logging.getLogger(__name__)
RUFF_LOG = "../logs/ruff_errors.log"
          
def ruff_try_format(file_path: str) -> bool:
    """Try to format with ruff. Return True if file likely formatted, False on failure."""
    try:
        # Preferred: ruff format (formatter). Try that first.
        cp = run_cmd(["ruff", "format", file_path])
        if cp.returncode == 0:
            return True

        # Fallback: try ruff check --fix
        cp2 = run_cmd(["ruff", "check", file_path, "--fix"])
        if cp2.returncode == 0:
            return True

        # Log both outputs for debugging
        with open(RUFF_LOG, "a", encoding="utf-8") as lf:
            lf.write(f"\n--- Ruff failure on: {file_path} ---\n")
            lf.write(f"format rc={cp.returncode}\nstdout:\n{cp.stdout}\nstderr:\n{cp.stderr}\n")
            lf.write(f"check--fix rc={cp2.returncode}\nstdout:\n{cp2.stdout}\nstderr:\n{cp2.stderr}\n")
        logger.warning(f"Ruff formatting failed for {file_path} (see {RUFF_LOG})")
        
        return False

    except FileNotFoundError:
        logger.warning("Ruff not found in PATH. Please install: pip install ruff")
        
        return False
    except Exception as e:
        logger.exception(f"Unexpected error running ruff on {file_path}: {e}")
        
        return False

def fallback_clean_code(code: str) -> str:
    """Simple safe cleaning: rstrip trailing spaces, convert tabs to 4 spaces, collapse multi blank lines."""
    cleaned_lines = []
    for line in code.splitlines():
        # preserve leading indent, normalize tabs
        leading = len(line) - len(line.lstrip("\t "))
        # keep original leading whitespace but replace tabs with 4 spaces in entire line
        line = line.replace("\t", "    ")
        # strip trailing spaces
        line = line.rstrip()
        # if line only whitespace -> become empty string
        if line.strip() == "":
            cleaned_lines.append("")
        else:
            cleaned_lines.append(line)
    # collapse consecutive blank lines >1 to a single blank
    result = []
    prev_blank = False
    for l in cleaned_lines:
        if l == "":
            if not prev_blank:
                result.append(l)
            prev_blank = True
        else:
            result.append(l)
            prev_blank = False
    return "\n".join(result)

def chunk_code_ast(code: str) -> Optional[List[str]]:
    """Try to parse AST and extract top-level function/class definitions as chunks.
       Return None if parse fails."""
    try:
        tree = ast.parse(code)
        chunks = []
        
        for node in tree.body:
            if isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef, ast.ClassDef)):
                seg = ast.get_source_segment(code, node)
                if seg:
                    chunks.append(seg)
                    
        # If no function/class found, return whole file as single chunk
        if not chunks:
            return [code.strip()]
        
        return [c.strip() for c in chunks if c.strip()]
    except SyntaxError:
        return None
    except Exception:
        return None

def chunk_code_regex(code: str) -> List[str]:
    """Fallback chunker: split whenever we see a line starting with def/class (simple heuristic)."""
    chunks = []
    cur = []
    
    for line in code.splitlines():
        if line.lstrip().startswith(("def ", "class ")):
            if cur:
                chunks.append("\n".join(cur).strip())
                cur = []
                
        cur.append(line)
        
    if cur:
        chunks.append("\n".join(cur).strip())
        
    # keep non-empty
    return [c for c in chunks if c]

def build_corpus(repo_path: str, 
                 output_jsonl: str = "python_corpus.jsonl", 
                 output_txt: str = "python_corpus.txt"):
    files = read_python_files(repo_path)
    logger.info(f"Found {len(files)} .py files (under size limit).")
    written = 0
    
    with open(output_jsonl, "w", encoding="utf-8") as out_jsonl, \
        open(output_txt, "w", encoding="utf-8") as out_txt:
        for f in files:
            path = f["path"]
            # try to format with ruff (safe: doesn't raise)
            formatted_ok = ruff_try_format(path)

            # read formatted if possible, else fallback to original cleaned content
            try:
                with open(path, "r", encoding="utf-8", errors="ignore") as fh:
                    formatted_code = fh.read()
                if not formatted_ok:
                    # even if file was changed by ruff partially, we still apply a safe final cleanup
                    formatted_code = fallback_clean_code(formatted_code)
            except Exception:
                formatted_code = fallback_clean_code(f["content"])

            # chunk via AST first, else regex fallback
            chunks = chunk_code_ast(formatted_code)
            if chunks is None:
                chunks = chunk_code_regex(formatted_code)
                if not chunks:
                    # as last resort, use whole file
                    chunks = [formatted_code.strip()]

            for chunk in chunks:
                # JSONL with tags
                rec = {"path": path, "text": f"<code> {chunk.strip()} </code>"}
                out_jsonl.write(json.dumps(rec, ensure_ascii=False) + "\n")

                # TXT with tags, keep line breaks or flatten?
                flat_chunk = chunk.strip().replace("\n", " ")  # flatten for tokenizer
                out_txt.write(f"<code> {flat_chunk} </code>\n")

                written += 1

repo_list = [d for d in os.listdir(github_repos_dir) if os.path.join(github_repos_dir, d) and os.path.isdir(os.path.join(github_repos_dir, d))]
repo_list

['3b1b_manim',
 'abi_screenshot-to-code',
 'ageitgey_face_recognition',
 'AntonOsika_gpt-engineer',
 'browser-use_browser-use',
 'commaai_openpilot',
 'deepseek-ai_DeepSeek-V3',
 'fastapi_fastapi',
 'FoundationAgents_MetaGPT',
 'labmlai_annotated_deep_learning_paper_implementations',
 'langflow-ai_langflow',
 'microsoft_markitdown',
 'nvbn_thefuck',
 'openai_whisper',
 'public-apis_public-apis',
 'sherlock-project_sherlock',
 'swisskyrepo_PayloadsAllTheThings',
 'Textualize_rich',
 'TheAlgorithms_Python',
 'Z4nzu_hackingtool']

In [11]:
output_jsonl = github_repos_dir / "corpus.jsonl"
output_txt = github_repos_dir / "corpus.txt"

for repo in repo_list:
    repo_path = os.path.join(github_repos_dir, repo)
    build_corpus(repo_path, output_jsonl, output_txt)

2025-09-29 09:34:02,390 INFO: Found 95 .py files (under size limit).
2025-09-29 09:34:07,972 INFO: Found 50 .py files (under size limit).
2025-09-29 09:34:10,334 INFO: Found 30 .py files (under size limit).
2025-09-29 09:34:11,977 INFO: Found 84 .py files (under size limit).
2025-09-29 09:34:17,019 INFO: Found 245 .py files (under size limit).
2025-09-29 09:34:29,565 INFO: Found 481 .py files (under size limit).
2025-09-29 09:34:49,894 INFO: Found 5 .py files (under size limit).
2025-09-29 09:34:51,665 INFO: Found 1136 .py files (under size limit).
2025-09-29 09:35:43,498 INFO: Found 890 .py files (under size limit).
<unknown>:99: SyntaxWarning: invalid escape sequence '\F'
<unknown>:361: SyntaxWarning: invalid escape sequence '\d'
2025-09-29 09:36:20,722 INFO: Found 227 .py files (under size limit).
<unknown>:1: SyntaxWarning: invalid escape sequence '\m'
<unknown>:73: SyntaxWarning: invalid escape sequence '\d'
<unknown>:93: SyntaxWarning: invalid escape sequence '\e'
<unknown>:1: Sy

In [12]:
repo_df = pd.read_json(output_jsonl, lines=True)

repo_data_cleaned = [item for item in repo_df['text'].tolist()]

repo_data_cleaned[:3]

['<code> def clear_screen():\n    os.system("cls" if system() == "Windows" else "clear") </code>',
 '<code> def validate_input(ip, val_range):\n    val_range = val_range or []\n    try:\n        ip = int(ip)\n        if ip in val_range:\n            return ip\n    except Exception:\n        return None\n    return None </code>',
 '<code> class HackingTool(object):\n    # About the HackingTool\n    TITLE: str = ""  # used to show info in the menu\n    DESCRIPTION: str = ""\n\n    INSTALL_COMMANDS: List[str] = []\n    INSTALLATION_DIR: str = ""\n\n    UNINSTALL_COMMANDS: List[str] = []\n\n    RUN_COMMANDS: List[str] = []\n\n    OPTIONS: List[Tuple[str, Callable]] = []\n\n    PROJECT_URL: str = ""\n\n    def __init__(self, options=None, installable: bool = True, runnable: bool = True):\n        options = options or []\n        if isinstance(options, list):\n            self.OPTIONS = []\n            if installable:\n                self.OPTIONS.append(("Install", self.install))\n         

In [13]:
def segment_sentences(text: str) -> List[str]:
    """ 
    Segment text into sentences based on punctuation.
    """
    sentences = re.split(r"(?<=[.!?])\s+", text)

    return [s.strip() for s in sentences if s.strip()]

def segment_paragraphs(text: str):
    """
    Segment text into paragraphs based on double newlines.
    """
    paragraphs = re.split(r'\n\s*\n', text.strip())
    return [p.strip() for p in paragraphs if p.strip()]

def chunk_with_tags(text, prefix="<wiki>", suffix="</wiki>", max_sentences=5):
    sentences = segment_sentences(text)
    chunks = []
    
    for i in range(0, len(sentences), max_sentences):
        chunk = " ".join(sentences[i:i+max_sentences]).strip()
        if chunk:
            chunks.append(f"{prefix} {chunk} {suffix}")
            
    return chunks

In [14]:
wiki_data_cleaned = [segment_sentences(item) for item in wiki_data_cleaned]
news_data_cleaned = [segment_sentences(item) for item in news_data_cleaned]

wiki_data_chunked = list(chain.from_iterable(
    chunk_with_tags(" ".join(item), prefix="<wiki>", suffix="</wiki>", max_sentences=5) 
    for item in wiki_data_cleaned
))

news_data_chunked = list(chain.from_iterable(
    chunk_with_tags(" ".join(item), prefix="<news>", suffix="</news>", max_sentences=5) 
    for item in news_data_cleaned
))

In [15]:
news_data_chunked[:3]

['<news> Title: Công an Hà Nội khuyến cáo người dân không tham gia một giải chạy do chưa đủ điều kiện pháp lý\nSection Header: Để bảo đảm an toàn cho bản thân và tuân thủ các quy định của pháp luật, Công an Thành phố Hà Nội khuyến cáo người dân không tham gia Giải chạy "Mỹ Đình Half Marathon 2025" vào ngày 28/9/2025. Section Content: Giải chạy "Mỹ Đình Half Marathon 2025" dự kiến diễn ra vào ngày 28/9/2025 tại khu vực Sân vận động Quốc gia Mỹ Đình chưa đủ điều kiện pháp lý để tổ chức. Ảnh minh họa \nSection Header: Giải chạy "Mỹ Đình Half Marathon 2025" chưa được chấp thuận về mặt chuyên môn và chưa được chấp thuận chủ trương tổ chức\nSection Content: Công an thành phố Hà Nội thông tin, qua công tác nắm tình hình, phòng An ninh chính trị nội bộ Công an thành phố Hà Nội phát hiện giải chạy "Mỹ Đình Half Marathon 2025" dự kiến diễn ra vào ngày 28/9/2025 tại khu vực Sân vận động Quốc gia Mỹ Đình chưa đủ điều kiện pháp lý để tổ chức. Cụ thể đến thời điểm hiện tại, Công ty Cổ phần Di sản Vă

In [16]:
def save_to_txt(list_data, file_path):
    """
    Save list of strings to a plain .txt file.
    Each element in list_data is written on a new line.
    """
    with open(file_path, "w", encoding="utf-8") as f:
        for item in list_data:
            f.write(item.strip() + "\n")


def save_to_jsonl(list_data, file_path):
    """
    Save list of strings to a .jsonl file.
    Each element in list_data is wrapped in {"text": ...}
    """
    with open(file_path, "w", encoding="utf-8") as f:
        for item in list_data:
            json_obj = {"text": item.strip()}
            f.write(json.dumps(json_obj, ensure_ascii=False) + "\n")
            
save_to_txt(wiki_data_chunked, wiki_data_dir / "wiki.txt")
save_to_txt(news_data_chunked, news_data_dir / "news.txt")

save_to_jsonl(wiki_data_chunked, wiki_data_dir / "wiki.jsonl")
save_to_jsonl(news_data_chunked, news_data_dir / "news.jsonl")

In [21]:
files = [
    str(wiki_data_dir / "wiki.txt"),
    str(news_data_dir / "news.txt"),
    str(output_txt)
]

print(files)

tokenizer = ByteLevelBPETokenizer()

# Train BPE
tokenizer.train(
    files=files,
    vocab_size=50000,
    min_frequency=3,
    special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>", "<wiki>", "<code>", "<news>"]
)

# save tokenizer
tokenizer.save_model("../data/processed/tokenizer")

['..\\data\\raw\\wiki\\wiki.txt', '..\\data\\raw\\news\\news.txt', '..\\data\\raw\\github_repos\\corpus.txt']


['../data/processed/tokenizer\\vocab.json',
 '../data/processed/tokenizer\\merges.txt']

In [22]:
enc = tokenizer.encode("Trí tuệ nhân tạo là một lĩnh vực...")
print(enc.tokens)
print(len(enc.tokens))

['Tr', 'ÃŃ', 'Ġtuá»ĩ', 'ĠnhÃ¢n', 'Ġtáº¡o', 'ĠlÃł', 'Ġmá»Ļt', 'ĠlÄ©nh', 'Ġvá»±c', '...']
10


In [23]:
enc = tokenizer.encode("def foo(): return 42")
print(enc.tokens)
print(len(enc.tokens))

['de', 'f', 'Ġf', 'oo', '():', 'Ġreturn', 'Ġ4', '2']
8


In [20]:
enc = tokenizer.encode("<code> def foo(): return 42 </code>")
print(enc.tokens)
print(len(enc.tokens))

['<code>', 'Ġdef', 'Ġf', 'oo', '():', 'Ġreturn', 'Ġ42', 'Ġ</', 'code', '>']
10
